# Paso 1: Importar Librerías de Python


In [ ]:
#pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163482 sha256=917cadd25e91fee782a300aaa0f470949e92bd921717eb90c4b83eaf35e67060
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
# Análisis de datos y manipulación
import pandas as pd
import numpy as np
import random

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Utilidades de fechas
from datetime import datetime

# Integración con Google Colab
from google.colab import drive

# Machine Learning: Preprocesamiento y división de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Machine Learning: Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

# Machine Learning: Modelado y validación
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

# Deep Learning: Construcción de modelos y capas
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout, Dot

# Deep Learning: Optimización y regularización
from keras.optimizers import Adam
from keras.regularizers import l2

# Deep Learning: Callbacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Paso 2: Lectura del Dataset

In [ ]:
# Suponiendo que tu DataFrame se llama df
df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')  # Descomenta y proporciona la ruta si estás cargando un archivo CSV

<ipython-input-6-1f8e3361d82a>:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')  # Descomenta y proporciona la ruta si estás cargando un archivo CSV


In [ ]:
df.shape

(4800571, 10)

# Modelo de Filtrado Colaborativo usando Embedding con Keras:

Funcionamiento:

Los embeddings son representaciones vectoriales densas y de baja dimensión de ítems y/o usuarios. Estas representaciones capturan relaciones semánticas entre ítems o entre usuarios.

Un método popular para generar embeddings es la factorización de matrices, como la descomposición en valores singulares (SVD). En el contexto de sistemas de recomendación, se busca factorizar la matriz de interacciones usuario-ítem en dos matrices más pequeñas (una para los usuarios y otra para los ítems) cuyo producto aproximado reproduce la matriz original lo mejor posible.

Una vez que se han obtenido los embeddings, la predicción de una calificación o interacción entre un usuario e ítem se realiza tomando el producto escalar entre sus embeddings respectivos.

Ventajas:

Simplicidad y eficiencia en términos computacionales.

Puede manejar grandes conjuntos de datos debido a su naturaleza de baja dimensión.

Es efectivo para capturar patrones subyacentes en los datos.

Desventajas:

No tiene en cuenta características adicionales de usuarios o ítems.

Dificultades para manejar nuevos ítems o usuarios (problema de arranque en frío).

El código ahora incluye regularización en los embeddings, un callback para detener el entrenamiento si no hay mejoras (early stopping) y otro para reducir la tasa de aprendizaje si el error de validación no mejora (reduce learning rate on plateau).

In [ ]:
data = df

In [ ]:
# Crear un LabelEncoder para cada columna
reviewerID_encoder = LabelEncoder()
asin_encoder = LabelEncoder()

# Ajustar y transformar las columnas
data['reviewerID_encoded'] = reviewerID_encoder.fit_transform(data['reviewerID'])
data['asin_encoded'] = asin_encoder.fit_transform(data['asin'])

# Número de usuarios únicos y número de ítems únicos
n_users = len(np.unique(data['reviewerID_encoded']))
n_items = len(np.unique(data['asin_encoded']))

# Dimensiones del embedding
embedding_dim = 10

# Entradas
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

# Embeddings con regularización
user_embedding = Embedding(n_users, embedding_dim, embeddings_regularizer=l2(1e-6))(user_input)
item_embedding = Embedding(n_items, embedding_dim, embeddings_regularizer=l2(1e-6))(item_input)

# Producto punto para predecir la valoración/rating
merged = Dot(axes=2)([user_embedding, item_embedding])
merged = Flatten()(merged)

# Modelo
model = Model(inputs=[user_input, item_input], outputs=merged)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

verbose=0: No muestra ninguna barra de progreso ni métricas.

verbose=1: Muestra una barra de progreso y actualiza las métricas después de cada lote.

verbose=2: Muestra las métricas después de cada época, pero no muestra la barra de progreso.

In [ ]:
from sklearn.model_selection import train_test_split

# Crear los arrays con los datos
user_ids = data['reviewerID_encoded'].values
item_ids = data['asin_encoded'].values
ratings = data['overall'].values

# División en entrenamiento y prueba
(user_ids_temp, user_ids_test, item_ids_temp, item_ids_test, ratings_temp, ratings_test) = train_test_split(
    user_ids, item_ids, ratings, test_size=0.2, random_state=42)

# División del conjunto de entrenamiento en entrenamiento y validación
(user_ids_train, user_ids_val, item_ids_train, item_ids_val, ratings_train, ratings_val) = train_test_split(
    user_ids_temp, item_ids_temp, ratings_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [ ]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [ ]:
%%time
# Entrenar el modelo con el conjunto de entrenamiento
model.fit([user_ids_train, item_ids_train], ratings_train, validation_data=([user_ids_val, item_ids_val], ratings_val),
          epochs=500, batch_size=400000, verbose=1, callbacks=[early_stop, reduce_lr])

Epoch 1/500
8/8 [==============================] - 6s 55ms/step - loss: 19.5202 - mae: 4.2598 - val_loss: 19.5449 - val_mae: 4.2634 - lr: 0.0010
Epoch 2/500
8/8 [==============================] - 0s 27ms/step - loss: 19.5182 - mae: 4.2596 - val_loss: 19.5445 - val_mae: 4.2634 - lr: 0.0010
Epoch 3/500
8/8 [==============================] - 0s 26ms/step - loss: 19.5158 - mae: 4.2594 - val_loss: 19.5440 - val_mae: 4.2633 - lr: 0.0010
Epoch 4/500
8/8 [==============================] - 0s 23ms/step - loss: 19.5132 - mae: 4.2591 - val_loss: 19.5434 - val_mae: 4.2633 - lr: 0.0010
Epoch 5/500
8/8 [==============================] - 0s 25ms/step - loss: 19.5103 - mae: 4.2588 - val_loss: 19.5428 - val_mae: 4.2633 - lr: 0.0010
Epoch 6/500
8/8 [==============================] - 0s 24ms/step - loss: 19.5069 - mae: 4.2585 - val_loss: 19.5419 - val_mae: 4.2632 - lr: 0.0010
Epoch 7/500
8/8 [==============================] - 0s 23ms/step - loss: 19.5029 - mae: 4.2581 - val_loss: 19.5409 - val_mae: 4.263

In [ ]:
y_true = ratings_test
predictions = model.predict([user_ids_test, item_ids_test])

30004/30004 [==============================] - 40s 1ms/step


In [ ]:
# Evaluar el modelo con el conjunto de prueba
loss, mae = model.evaluate([user_ids_test, item_ids_test], ratings_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test MAE: {mae:.4f}')

30004/30004 [==============================] - 78s 3ms/step - loss: 19.4828 - mae: 3.8881
Test Loss: 19.4828
Test MAE: 3.8881


In [ ]:
# MSE
mse = mean_squared_error(y_true, predictions)
print(f"MSE: {mse:.4f}")

# RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

# MAE
mae = mean_absolute_error(y_true, predictions)
print(f"MAE: {mae:.4f}")

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((y_true - predictions.flatten()) / y_true)) * 100
print(f"MAPE: {mape:.2f}%")

MSE: 17.8720
RMSE: 4.2275
MAE: 3.8881
MAPE: 91.58%


# Recomedanción

In [ ]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id_test = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Identificar los ítems que el usuario ha calificado en el conjunto de prueba
indices_usuario = np.where(user_ids_test == usuario_id_test_encoded)[0]
items_calificados_test_encoded = item_ids_test[indices_usuario]
calificaciones_reales_test = ratings_test[indices_usuario]

# Predecir las calificaciones para estos ítems
calificaciones_predichas = model.predict([np.array([usuario_id_test_encoded] * len(items_calificados_test_encoded)), items_calificados_test_encoded])

# Decodificar los ítems para obtener los ID originales
items_calificados_test = asin_encoder.inverse_transform(items_calificados_test_encoded)

# Comparar las calificaciones predichas con las reales
comparacion = list(zip(items_calificados_test, calificaciones_reales_test, calificaciones_predichas.flatten()))

# Mostrar las comparaciones
print(f"Recomendaciones para el usuario {usuario_id_test}:")
for item, real, predicha in comparacion:
    print(f"Ítem: {item}, Calificación Real: {real}, Calificación Predicha: {predicha}")


1/1 [==============================] - 0s 33ms/step
Recomendaciones para el usuario A15AE3607YBHF6:
Ítem: B013GVHED0, Calificación Real: 5.0, Calificación Predicha: -0.316520631313324


In [ ]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Lista de todos los ítems únicos en el dataset
items_unicos = np.unique(data['asin'].values)

# Ítems que el usuario ya ha calificado
items_calificados = data[data['reviewerID_encoded'] == usuario_id_test_encoded]['asin'].tolist()

# Ítems para predecir
items_a_predecir = [item for item in items_unicos if item not in items_calificados]

# Convierte los ítems a su representación codificada
items_a_predecir_encoded = asin_encoder.transform(items_a_predecir)

# Predecir la calificación para cada ítem no calificado
predicciones = model.predict([np.array([usuario_id_test_encoded] * len(items_a_predecir_encoded)), items_a_predecir_encoded])

# Emparejar cada ítem con su predicción
items_predicciones = list(zip(items_a_predecir, predicciones.flatten()))

# Ordenar por la calificación predicha más alta
items_recomendados = sorted(items_predicciones, key=lambda x: x[1], reverse=True)

# Tomar los top N recomendaciones
top_n = 5
top_recomendaciones = items_recomendados[:top_n]

# Imprimir las recomendaciones
print(f"Recomendaciones para el usuario {usuario_id}:")
for item, calificacion in top_recomendaciones:
    print(f"Ítem: {item}, Calificación Predicha: {calificacion}")

4814/4814 [==============================] - 6s 1ms/step
Recomendaciones para el usuario AHWJ5Z2QZU97:
Ítem: B000KR1D1S, Calificación Predicha: 2.7065205574035645
Ítem: B00LMNKEPK, Calificación Predicha: 2.692791223526001
Ítem: B004L8J15C, Calificación Predicha: 2.628918170928955
Ítem: B0023TRMM0, Calificación Predicha: 2.61202073097229
Ítem: B005ZAKR1C, Calificación Predicha: 2.5624284744262695


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa con Keras:

Funcionamiento:

Estos sistemas toman características de los ítems y/o usuarios y las pasan a través de una o varias capas de neuronas para obtener una predicción.
Las redes neuronales son capaces de capturar interacciones no lineales entre características, lo que las hace poderosas para tareas de modelado complejas.

Ventajas:

Capacidad de modelar relaciones no lineales.

Flexibilidad para incorporar múltiples fuentes de datos o características.

Puede manejar arranques en frío al incorporar características de nuevos ítems o usuarios.

Desventajas:

Mayor costo computacional en comparación con los sistemas basados únicamente en embeddings.

Riesgo de sobreajuste si no se tiene un conjunto de datos lo suficientemente grande.

Ahora el código incluye regularización en los embeddings, early stopping para detener el entrenamiento si el modelo deja de mejorar en el conjunto de validación, y reducción de la tasa de aprendizaje si el error en el conjunto de validación no mejora después de algunas épocas. Estas adiciones deberían ayudar a mejorar la capacidad de generalización del modelo.

In [ ]:
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
hidden_units = [128, 64, 32]
dropout_rate = 0.2
l2_reg = 1e-6

# Arquitectura del modelo
user_input = Input(shape=[1], name='user_input')
item_input = Input(shape=[1], name='item_input')
concat = Concatenate()([user_input, item_input])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu', kernel_regularizer=l2(l2_reg))(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1, kernel_regularizer=l2(l2_reg))(dense)  # También puedes aplicar regularización L2 aquí
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

In [ ]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [ ]:
# Datos para entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# División en entrenamiento y prueba
(user_ids_temp, user_ids_test, item_ids_temp, item_ids_test, ratings_temp, ratings_test) = train_test_split(
    user_ids, item_ids, ratings, test_size=0.2, random_state=42)

# División del conjunto de entrenamiento en entrenamiento y validación
(user_ids_train, user_ids_val, item_ids_train, item_ids_val, ratings_train, ratings_val) = train_test_split(
    user_ids_temp, item_ids_temp, ratings_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [ ]:
%%time
# Entrenamiento del modelo
history = model.fit([user_ids_train, item_ids_train], ratings_train,
    epochs=500,validation_data=([user_ids_val, item_ids_val], ratings_val),
    batch_size=400000,verbose=1,callbacks=[early_stop, reduce_lr])

Epoch 1/500
8/8 [==============================] - 3s 80ms/step - loss: 2217572608.0000 - val_loss: 31106164.0000 - lr: 0.0010
Epoch 2/500
8/8 [==============================] - 0s 52ms/step - loss: 820925504.0000 - val_loss: 122897216.0000 - lr: 0.0010
Epoch 3/500
8/8 [==============================] - 0s 52ms/step - loss: 461750208.0000 - val_loss: 746341.8125 - lr: 0.0010
Epoch 4/500
8/8 [==============================] - 0s 52ms/step - loss: 256133584.0000 - val_loss: 10175531.0000 - lr: 0.0010
Epoch 5/500
8/8 [==============================] - 0s 52ms/step - loss: 157639536.0000 - val_loss: 14076202.0000 - lr: 0.0010
Epoch 6/500
8/8 [==============================] - 0s 52ms/step - loss: 109848368.0000 - val_loss: 3472343.0000 - lr: 0.0010
Epoch 7/500
8/8 [==============================] - 0s 52ms/step - loss: 79891592.0000 - val_loss: 11297249.0000 - lr: 0.0010
Epoch 8/500
8/8 [==============================] - 0s 53ms/step - loss: 58523760.0000 - val_loss: 13522179.0000 - lr: 0.

In [ ]:
# Evaluar el modelo
y_true = ratings_test
predictions = model.predict([user_ids_test, item_ids_test])

30004/30004 [==============================] - 46s 2ms/step


In [ ]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# Funciones MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 6.837962360131579
RMSE: 2.61494978156973
MAE: 2.2266781262002002
MAPE: 56.486429627232916%


# Recomendación

In [ ]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id_test = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Identificar los ítems que el usuario ha calificado en el conjunto de prueba
indices_usuario = np.where(user_ids_test == usuario_id_test_encoded)[0]
items_calificados_test_encoded = item_ids_test[indices_usuario]
calificaciones_reales_test = ratings_test[indices_usuario]

# Predecir las calificaciones para estos ítems
calificaciones_predichas = model.predict([np.array([usuario_id_test_encoded] * len(items_calificados_test_encoded)), items_calificados_test_encoded])

# Decodificar los ítems para obtener los ID originales
items_calificados_test = asin_encoder.inverse_transform(items_calificados_test_encoded)

# Comparar las calificaciones predichas con las reales
comparacion = list(zip(items_calificados_test, calificaciones_reales_test, calificaciones_predichas.flatten()))

# Mostrar las comparaciones
print(f"Recomendaciones para el usuario {usuario_id_test}:")
for item, real, predicha in comparacion:
    print(f"Ítem: {item}, Calificación Real: {real}, Calificación Predicha: {predicha}")

1/1 [==============================] - 0s 22ms/step
Recomendaciones para el usuario A3EJ3CQXPQAAWY:
Ítem: B003H5YNVC, Calificación Real: 1.0, Calificación Predicha: 3.009850263595581


In [ ]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Lista de todos los ítems únicos en el dataset
items_unicos = np.unique(data['asin'].values)

# Ítems que el usuario ya ha calificado
items_calificados = data[data['reviewerID_encoded'] == usuario_id_test_encoded]['asin'].tolist()

# Ítems para predecir
items_a_predecir = [item for item in items_unicos if item not in items_calificados]

# Convierte los ítems a su representación codificada
items_a_predecir_encoded = asin_encoder.transform(items_a_predecir)

# Predecir la calificación para cada ítem no calificado
predicciones = model.predict([np.array([usuario_id_test_encoded] * len(items_a_predecir_encoded)), items_a_predecir_encoded])

# Emparejar cada ítem con su predicción
items_predicciones = list(zip(items_a_predecir, predicciones.flatten()))

# Ordenar por la calificación predicha más alta
items_recomendados = sorted(items_predicciones, key=lambda x: x[1], reverse=True)

# Tomar los top N recomendaciones
top_n = 5
top_recomendaciones = items_recomendados[:top_n]

# Imprimir las recomendaciones
print(f"Recomendaciones para el usuario {usuario_id}:")
for item, calificacion in top_recomendaciones:
    print(f"Ítem: {item}, Calificación Predicha: {calificacion}")

4814/4814 [==============================] - 7s 2ms/step
Recomendaciones para el usuario A25WRT7SHNQ5TK:
Ítem: 0871167042, Calificación Predicha: 1.4638738632202148
Ítem: 1519588135, Calificación Predicha: 1.4638735055923462
Ítem: 1579652956, Calificación Predicha: 1.463873267173767
Ítem: 1936023857, Calificación Predicha: 1.463873028755188
Ítem: 5120053017, Calificación Predicha: 1.4638727903366089


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa y Embedding con Keras:

Otra enfoque popular para filtrado colaborativo es utilizar redes neuronales multicapa, que esencialmente aprenden características no lineales de los datos. Vamos a construir un modelo que fusiona los embeddings de usuarios y artículos (por ejemplo, películas) en una red neuronal densa.

Este enfoque, que combina embeddings y redes neuronales multicapa, puede capturar interacciones más complejas y no lineales entre usuarios y artículos. Sin embargo, es crucial prestar atención al sobreajuste y asegurarse de que el modelo no esté simplemente memorizando los datos. Por lo tanto, es recomendable emplear técnicas de regularización, ajustar hiperparámetros y validar el rendimiento con un conjunto de datos de validación.

Funcionamiento:

Combina lo mejor de ambos mundos. Primero, se utilizan embeddings para convertir ítems y usuarios en representaciones vectoriales densas. Luego, estas representaciones se pasan a través de una red neuronal para hacer la predicción.

El proceso generalmente comienza con capas de embedding que convierten identificadores de usuarios e ítems en vectores. Estos vectores luego se pasan a través de capas densas para obtener la predicción final.

Ventajas:

Capacidad de capturar patrones subyacentes en los datos mediante embeddings y modelar interacciones no lineales mediante la red neuronal.

Flexibilidad para incorporar características adicionales.

Potencialmente más preciso que cualquiera de los otros dos métodos por separado.

Desventajas:

Mayor complejidad en el modelado y entrenamiento.

Mayor costo computacional.

Requiere un ajuste más cuidadoso y riesgo de sobreajuste si no se gestiona adecuadamente.

Regularización L2: Agregar una regularización L2 a las capas de embedding y densas para prevenir el sobreajuste.

Early Stopping: Añadir una callback de early stopping para detener el entrenamiento cuando no haya mejora en el conjunto de validación durante un cierto número de épocas.

Ajuste de la tasa de aprendizaje: Utilizar la callback ReduceLROnPlateau para reducir la tasa de aprendizaje cuando no haya mejora en el conjunto de validación.

Métricas adicionales: Puedes monitorizar otras métricas como el error absoluto medio (MAE) durante el entrenamiento.

In [ ]:
# Convertir reviewerID y asin a índices numéricos secuenciales
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
n_latent_factors = 50
hidden_units = [128, 64]
dropout_rate = 0.2
l2_reg = 1e-4

# Arquitectura del modelo con regularización L2
user_embedding = Embedding(n_users, n_latent_factors, embeddings_regularizer=l2(l2_reg), name='user_embedding')(user_input)
item_embedding = Embedding(n_items, n_latent_factors, embeddings_regularizer=l2(l2_reg), name='item_embedding')(item_input)

user_vec = Flatten(name='flatten_users')(user_embedding)
item_vec = Flatten(name='flatten_items')(item_embedding)
concat = Concatenate()([user_vec, item_vec])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu', kernel_regularizer=l2(l2_reg))(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1, kernel_regularizer=l2(l2_reg))(dense)
model = Model(inputs=[user_input, item_input], outputs=output)

model.compile(optimizer=Adam(0.001), loss='mean_squared_error', metrics=['mae'])

In [ ]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, min_lr=1e-5)

In [ ]:
# Entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# División en entrenamiento y prueba
(user_ids_temp, user_ids_test, item_ids_temp, item_ids_test, ratings_temp, ratings_test) = train_test_split(
    user_ids, item_ids, ratings, test_size=0.2, random_state=42)

# División del conjunto de entrenamiento en entrenamiento y validación
(user_ids_train, user_ids_val, item_ids_train, item_ids_val, ratings_train, ratings_val) = train_test_split(
    user_ids_temp, item_ids_temp, ratings_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [ ]:
%%time
history = model.fit([user_ids_train, item_ids_train], ratings_train,
                    epochs=500,validation_data=([user_ids_val, item_ids_val], ratings_val),
                    batch_size=400000,verbose=1,callbacks=[early_stop, reduce_lr])

Epoch 1/500
8/8 [==============================] - 3s 211ms/step - loss: 24.7587 - mae: 4.1822 - val_loss: 22.5928 - val_mae: 4.0783 - lr: 0.0010
Epoch 2/500
8/8 [==============================] - 1s 136ms/step - loss: 21.1200 - mae: 3.9819 - val_loss: 18.7639 - val_mae: 3.8057 - lr: 0.0010
Epoch 3/500
8/8 [==============================] - 1s 136ms/step - loss: 16.8780 - mae: 3.6106 - val_loss: 13.7046 - val_mae: 3.2457 - lr: 0.0010
Epoch 4/500
8/8 [==============================] - 1s 135ms/step - loss: 11.0200 - mae: 2.8626 - val_loss: 7.0080 - val_mae: 2.2075 - lr: 0.0010
Epoch 5/500
8/8 [==============================] - 1s 137ms/step - loss: 4.7137 - mae: 1.6400 - val_loss: 3.0387 - val_mae: 1.0708 - lr: 0.0010
Epoch 6/500
8/8 [==============================] - 1s 137ms/step - loss: 3.3119 - mae: 1.1445 - val_loss: 2.8788 - val_mae: 1.0109 - lr: 0.0010
Epoch 7/500
8/8 [==============================] - 1s 134ms/step - loss: 2.4330 - mae: 0.9451 - val_loss: 2.2958 - val_mae: 1.017

In [ ]:
# Predicciones
y_true = ratings_test
predictions = model.predict([user_ids_test, item_ids_test])

30004/30004 [==============================] - 45s 2ms/step


In [ ]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 1.364365152190864
RMSE: 1.1680604231763287
MAE: 0.8237411169192218
MAPE: 41.29704997473233%


# Recomendación

In [ ]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id_test = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Identificar los ítems que el usuario ha calificado en el conjunto de prueba
indices_usuario = np.where(user_ids_test == usuario_id_test_encoded)[0]
items_calificados_test_encoded = item_ids_test[indices_usuario]
calificaciones_reales_test = ratings_test[indices_usuario]

# Predecir las calificaciones para estos ítems
calificaciones_predichas = model.predict([np.array([usuario_id_test_encoded] * len(items_calificados_test_encoded)), items_calificados_test_encoded])

# Decodificar los ítems para obtener los ID originales
items_calificados_test = asin_encoder.inverse_transform(items_calificados_test_encoded)

# Comparar las calificaciones predichas con las reales
comparacion = list(zip(items_calificados_test, calificaciones_reales_test, calificaciones_predichas.flatten()))

# Mostrar las comparaciones
print(f"Recomendaciones para el usuario {usuario_id_test}:")
for item, real, predicha in comparacion:
    print(f"Ítem: {item}, Calificación Real: {real}, Calificación Predicha: {predicha}")

1/1 [==============================] - 0s 23ms/step
Recomendaciones para el usuario A1S5BPHBX5DQ0X:
Ítem: B00BBZOWXE, Calificación Real: 5.0, Calificación Predicha: 4.3351616859436035
Ítem: B000XDJ7LW, Calificación Real: 4.0, Calificación Predicha: 4.001919269561768


In [ ]:
# Seleccionar un usuario al azar del conjunto de prueba
usuario_id_test_encoded = random.choice(user_ids_test)

# Decodificar el ID del usuario para obtener el ID original
usuario_id = reviewerID_encoder.inverse_transform([usuario_id_test_encoded])[0]

# Lista de todos los ítems únicos en el dataset
items_unicos = np.unique(data['asin'].values)

# Ítems que el usuario ya ha calificado
items_calificados = data[data['reviewerID_encoded'] == usuario_id_test_encoded]['asin'].tolist()

# Ítems para predecir
items_a_predecir = [item for item in items_unicos if item not in items_calificados]

# Convierte los ítems a su representación codificada
items_a_predecir_encoded = asin_encoder.transform(items_a_predecir)

# Predecir la calificación para cada ítem no calificado
predicciones = model.predict([np.array([usuario_id_test_encoded] * len(items_a_predecir_encoded)), items_a_predecir_encoded])

# Emparejar cada ítem con su predicción
items_predicciones = list(zip(items_a_predecir, predicciones.flatten()))

# Ordenar por la calificación predicha más alta
items_recomendados = sorted(items_predicciones, key=lambda x: x[1], reverse=True)

# Tomar los top N recomendaciones
top_n = 5
top_recomendaciones = items_recomendados[:top_n]

# Imprimir las recomendaciones
print(f"Recomendaciones para el usuario {usuario_id}:")
for item, calificacion in top_recomendaciones:
    print(f"Ítem: {item}, Calificación Predicha: {calificacion}")

4814/4814 [==============================] - 8s 2ms/step
Recomendaciones para el usuario A1X08FQ482BNFQ:
Ítem: B019HIY8QC, Calificación Predicha: 4.775868892669678
Ítem: B00SKFUR2S, Calificación Predicha: 4.702950477600098
Ítem: B0172XZO12, Calificación Predicha: 4.6992034912109375
Ítem: B00TXUXFE6, Calificación Predicha: 4.673807621002197
Ítem: B00PWYSGVO, Calificación Predicha: 4.664052963256836


Estas métricas te ayudarán a tener una mejor idea del desempeño de tu modelo. Por ejemplo:

MSE y RMSE son útiles cuando quieres penalizar grandes errores.

MAE te da una idea del error medio sin considerar la dirección del error.

MAPE es útil cuando quieres representar el error en términos porcentuales.

Para una evaluación completa, es recomendable utilizar un conjunto de validación aparte (es decir, no solo depender del validation_split). Esto asegura que estás evaluando el desempeño en datos que el modelo nunca ha visto durante el entrenamiento.

En resumen, la elección del tipo de sistema de recomendación dependerá de la naturaleza del conjunto de datos, las características disponibles, las capacidades computacionales y el tipo de relaciones o interacciones que se desean capturar. A menudo, un enfoque híbrido (como el sistema que combina embeddings y redes neuronales) ofrece un buen equilibrio entre precisión y eficiencia.